In [4]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/ 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [7]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np

In [8]:
file = "/home/isidro/Desktop/chains_server/chains_1000/chains/owaCDM_phy_HD+SN+BBAO+Planck_nested_dynesty_multi_1.txt"

split = 0.8
numNeurons = 500
epochs = 300
nlayers = 2
batch_size = 0

params = np.loadtxt(file, usecols=(2,3,4,5,6))
logL = np.loadtxt(file, usecols=(1))

nparams = len(params)
randomize = np.random.permutation(nparams)
params = params[randomize]
logL = logL[randomize]

maxLogL = np.max(logL)
minLogL = np.min(logL)
ntrain = int(split * nparams)
indx = [ntrain]
# xtrain <- params_training ytrain<-logLtraining
params_training, params_testing = np.split(params, indx)
logL_training, logL_testing = np.split(logL, indx)

In [20]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=1e-3,
                                   patience=5,
                                   restore_best_weights=True)]

n_cols = params_training.shape[1]

In [10]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))


In [19]:
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric('loss', display_name="Loss")])  


In [39]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu, input_shape=(n_cols,)),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        tf.keras.layers.Dense(1)
    ])
                              
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='mean_squared_error', metrics=['mean_squared_error']
      )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model.fit(params_training, logL_training, epochs=100, callbacks=callbacks) 
    _, loss = model.evaluate(params_testing, logL_testing)
    
    
    return loss

In [40]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    loss = train_test_model(hparams)
    tf.summary.scalar("loss", loss, step=1)


In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/100
557/557 [==============================] - 0s 679us/step - loss: 6690221056.0000 - mean_squared_error: 6690221056.0000
Epoch 2/100
557/557 [==============================] - 0s 746us/step - loss: 6624987648.0000 - mean_squared_error: 6624987648.0000
Epoch 3/100
557/557 [==============================] - 0s 841us/step - loss: 6561214464.0000 - mean_squared_error: 6561214464.0000
Epoch 4/100
557/557 [==============================] - 0s 776us/step - loss: 6539017216.0000 - mean_squared_error: 6539017216.0000
Epoch 5/100
557/557 [==============================] - 0s 838us/step - loss: 6522632192.0000 - mean_squared_error: 6522632192.0000
Epoch 6/100
557/557 [==============================] - 0s 794us/step - loss: 6505781248.0000 - mean_squared_error: 6505781248.0000
Epoch 7/100
557/557 [==============================] - 1s 932us/step - loss: 6489205248.0000 - mean_squared_error: 6489205248.0000
Ep